In [1]:
import os
import pandas as pd
import glob
from tqdm import tqdm
from scipy.io import wavfile
from scipy import signal
import numpy as np
import soundfile as sf
os.chdir('../src/')
import config
import matplotlib.pyplot as plt
from tqdm import tqdm
import IPython.display as ipd
import json
import img.augmentation as aug
from sklearn.metrics import accuracy_score

In [2]:
def read_test_predictions(folder):
    predictions = pd.read_csv(config.OUTPUT_FOLDER + '%s/pred_test.csv'%folder)
    predictions['target'] = predictions.drop('id', 1).idxmax(axis = 1)
    predictions["prob_%s"%folder] = predictions[[str(x) for x in np.arange(32)]].max(axis = 1)
    predictions['target'] = predictions['target'].map({str(v):k for k,v in config.mapping_dict.iteritems()})
    predictions.rename(columns = {'target' : 'target_%s'%folder}, inplace = True)
    gt_test = pd.read_csv(config.DATA_FOLDER + "gt_test.csv")
    predictions = predictions.merge(gt_test, on = 'id')
    assert len(predictions) == len(gt_test)
    
    return predictions[['id', 'target_%s'%folder, "prob_%s"%folder]]

In [7]:
nn1 = read_test_predictions("nn_16")
nn2 = read_test_predictions("nn_19")

df = nn1.merge(nn2, on = "id")
assert len(df) == len(nn1)

In [4]:
df['is_unknown'] = 0
df.loc[(df['target_nn_16'] =="unknown") & (df['target_nn_19'] =="unknown"), "is_unknown"] = 1
df["prob"] = (df["prob_nn_16"] + df["prob_nn_19"]) / 2.0

In [63]:
paths = df[(df["is_unknown"] == 1) & (df["prob"] > 0.5)].sort_values("prob", ascending = False)["id"].tolist()

In [320]:
len(paths)

3090

In [321]:
with open(config.DATA_FOLDER + "unknown_unknown_nn16_nn19_3090.json", "w") as f:
    json.dump({"id" : paths}, f)

In [323]:
N = -180
name = str(paths[N])
path = "/home/tony/ssd_buffer/kaggle_speech/test/audio/%s.wav"%name
ipd.Audio(path)

In [308]:
sub53 = pd.read_csv("/home/tony/Downloads/submit53.csv")

In [303]:
zz['label'].value_counts()

unknown    2374
on          217
right       118
go           96
left         76
down         69
no           44
up           36
off          32
stop         18
yes           9
silence       1
Name: label, dtype: int64

In [309]:
zz = sub53[sub53["fname"].isin([x + ".wav" for x in paths])]
zz_list = zz[zz["label"] == "right"]['fname'].tolist()

In [316]:
N = 110

name = str(zz_list[N])
path = "/home/tony/ssd_buffer/kaggle_speech/test/audio/%s"%name
ipd.Audio(path)

In [297]:
sub53['label'].value_counts()

unknown    77631
silence    14972
on          7422
no          7089
right       6914
left        6835
go          6532
off         6490
down        6451
up          6208
stop        6073
yes         5921
Name: label, dtype: int64

In [300]:
sub53.loc[sub53["fname"].isin([x + ".wav" for x in paths]),'label'] = "unknown"

In [301]:
sub53['label'].value_counts()

unknown    78347
silence    14971
on          7205
no          7045
right       6796
left        6759
off         6458
go          6436
down        6382
up          6172
stop        6055
yes         5912
Name: label, dtype: int64

In [302]:
sub53.to_csv("sub53_3090unknowns_from_nn16_nn19.csv", index = False)

In [ ]:
dif_df = nn2['target_nn_9'].value_counts().to_frame().reset_index().merge(nn1['target_nn_19'].value_counts().to_frame().reset_index(), on = "index", how = "right")
dif_df["dif"] = np.abs(dif_df['target_nn_9'] - dif_df['target_nn_19'])
dif_df

In [ ]:
paths = df[df['target_nn_19'] == 'up']['id'].tolist()

In [ ]:
paths[345]

In [ ]:
N = 123
name = paths[N]
path = "/home/tony/ssd_buffer/kaggle_speech/test/audio/%s.wav"%name
ipd.Audio(name)

In [ ]:
def read_train_predictions(folder):
    predictions = pd.read_csv(config.OUTPUT_FOLDER + '%s/pred_train.csv'%folder)
    predictions['target'] = predictions.drop('id', 1).idxmax(axis = 1)
    predictions['target'] = predictions['target'].map({str(v):k for k,v in config.mapping_dict.iteritems()})
    predictions.rename(columns = {'target' : 'target_%s'%folder}, inplace = True)
    gt_train = pd.read_csv(config.DATA_FOLDER + "gt_train.csv")
    predictions = predictions.merge(gt_train, on = 'id', how = 'left')
#     assert len(predictions) == len(gt_test)
    
    return predictions[['id', 'target', 'target_%s'%folder]]

nn16_train = read_train_predictions("nn_16")

In [ ]:
nn16_train.info()

In [ ]:
nn16_train_notnull = nn16_train[nn16_train['target'].notnull()]
accuracy_score(nn16_train_notnull['target'].values, nn16_train_notnull['target_nn_16'].values)

In [ ]:
nn16_train[nn16_train['target'].isnull()]['target_nn_16'].value_counts()

In [8]:
name = "clip_5bdc4d60c"
path = "/home/tony/ssd_buffer/kaggle_speech/test/audio/%s.wav"%name
ipd.Audio(path)

In [ ]:
paths = predictions[predictions['target'] == 'unknown']['path'].tolist()

In [ ]:
path

In [ ]:
path

In [ ]:
ipd.Audio(paths[1203])

In [ ]:
["follow", "visual", "forward", "backward", "learn", "? wareshop" , "? took"]

generate sub

In [ ]:
predictions['fname'] = predictions['id'] + '.wav'
predictions['label'] = predictions['target']
predictions['label'] = predictions['label'] .apply(lambda x: "hui" if x in config.allowed_train_labels else "unknown")
# predictions[['fname', 'label']].to_csv("3folds_of_nn16.csv", index = False)

In [ ]:
predictions[['label', 'target']].drop_duplicates(['label', 'target'])

In [ ]:
predictions.rename(columns = {'label' : "pred_label"}, inplace = True)

In [ ]:
submit53 = pd.read_csv("/home/tony/Downloads/submit53.csv")
submit53.rename(columns = {'label' : 'label_sub53'}, inplace = True)

In [ ]:
new_sub = submit53.merge(predictions[['fname', 'pred_label']], on = 'fname')
assert len(new_sub) == len(submit53)

In [ ]:
new_sub['label'] = '_lol_'
new_sub.loc[new_sub['pred_label'] == 'unknown', 'label'] = 'unknown'
new_sub.loc[new_sub['pred_label'] == 'hui', 'label'] = new_sub[new_sub['pred_label'] == 'hui']['label_sub53']

In [ ]:
new_sub[['fname', 'label']].to_csv('sub53_with_double_words.csv', index = False)

In [ ]:
new_sub[(new_sub['label_sub53']!= new_sub['pred_label']) & (new_sub['pred_label'] == "unknown")]

In [ ]:
sub = pd.read_csv(config.OUTPUT_FOLDER + "nn_19/pred_test.csv")

In [ ]:
sub['label'] = sub.drop("id", 1).idxmax(axis = 1)
sub['label'] = sub['label'].map({str(v) :k for k,v in config.mapping_dict.iteritems()})
sub['label'] = sub['label'].apply(lambda x: x if x in config.allowed_train_labels else "unknown")
sub['fname'] = sub['id'] + '.wav'

In [ ]:
sub[['fname', 'label']].to_csv(config.OUTPUT_FOLDER + "nn_19/nn_19.csv", index = False)

In [ ]:
gt_test = pd.read_csv(config.DATA_FOLDER  + "gt_test.csv")

In [ ]:
sub[~sub['id'].isin(gt_test['id'].tolist())]['label'].value_counts()

In [ ]:
sub2 = pd.read_csv(config.OUTPUT_FOLDER + "stacker_xgb_11/sub_test.csv")

In [ ]:
sub['label'].value_counts()

In [ ]:
sub['target'].value_counts()